This notebook is to construct `Replaces` objects out of text pairs we have saved before.

Not much comments here so one is necessary to look into `../replaces.py` file (bad idea probably).

In [ ]:
PATHES = {
    "librusec": {
        "input": "/home/jovyan/data/librusec_pairs.jsonl",
        "output": "/home/jovyan/data/librusec_replaces.jsonl",
    },
    "ficbook": {
        "input": "/home/jovyan/data/ficbook_pairs.jsonl",
        "output": "/home/jovyan/data/ficbook_replaces.jsonl",
    },
    "pikabu": {
        "input": "/home/jovyan/data/pikabu_pairs.jsonl",
        "output": "/home/jovyan/data/pikabu_replaces.jsonl",
    },
}

I use `Replaces` class as list of changes have been made upon the text.

In [ ]:
import os
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd())))

In [ ]:
from replaces import Replace, Replaces


Replaces.from_sequences(
    "мама мыла раму с мылом".split(),
    "мама раму уронила".split(),
    False
)

One is possible to construct Replaces object out from list of dicts so some kind of serialization could be done easily here

In [ ]:
Replaces([{"text_from": "a", "text_to": "a"}, {"text_from": "b", "text_to": "c"}])

In [ ]:
DATASET = "librusec"
# DATASET = "ficbook"
# DATASET = "pikabu"

In [ ]:
import json
from tqdm.auto import tqdm


with open(PATHES[DATASET]["input"]) as f:
    data = [json.loads(line) for line in tqdm(f)]

In [ ]:
import re


re_tokens = re.compile(r"[а-яА-ЯёЁ]+\s*|[a-zA-Z]+\s*|\d+(?:\.\d+)?\s*|[^а-яА-ЯёЁa-zA-Z\d\s]+\s*")


def tokenize(text):
    return re.findall(re_tokens, text)


"|".join(tokenize("ты, да я, да мы c тобой - вместе 2.5°C. C'est la vie! iPhone 10C pro15 f2f."))

In [ ]:
for elem in tqdm(data):
    if "replaces" not in elem:
        elem["replaces"] = Replaces.from_sequences(tokenize(elem["itn"]), tokenize(elem["tn"]))
        for r1, r2 in zip(elem["replaces"], elem["replaces"][1:]):
            if r1.type != "E" and r1.text_from.endswith(" ") and r1.text_to.endswith(" "):
                r1.text_from = r1.text_from[:-1]
                r1.text_to = r1.text_to[:-1]
                r2.text_from = " " + r2.text_from
                r2.text_to = " " + r2.text_to
data[0]

In [ ]:
for i, elem in enumerate(data[:10]):
    print(f'{i}\n{elem["replaces"]}\n')

In [ ]:
import json


with open(PATHES[DATASET]["output"], "w") as f:
    for elem in tqdm(data):
        json.dump({"replaces": elem["replaces"]}, f, ensure_ascii=False)
        f.write("\n")

In [ ]:
data[1]